# baseline0_itemCF

In [78]:
!python baseline0_itemCF.py

gen train_data_matrix begin...
cal cosine_similarity begin...
cal cosine_similarity end
100%|████████████████████████████████████| 27195/27195 [00:45<00:00, 600.60it/s]
935.4970018863678s
complete!!!


# baseline1_itemCF

In [13]:
!python baseline1_itemCF.py

phase: 0
phase: 1
phase: 2
100%|███████████████████████████████████████| 5079/5079 [01:00<00:00, 83.81it/s]
79.88833403587341s
complete!!!


# 思路总结
## baseline0_itemCF
通过建立 user-item矩阵，通过用户的点击行为得出item的相似性，如item1和item2都只被u1和u2各点击过1次，那么item1和item2相似度为1

推荐过程:
已知u1点击了i1，那么拿到i1最相近的前50个item推荐


## baseline1_itemCF
通过建立 item-item共现矩阵，通过item1和item2被同一user同时点击来计算item的相似性，如u1点击过i1和i2，则对i1和i2共现次数贡献1，如果u2也点击过i1和i2，则对i1和i2共现次数贡献+1

推荐过程：
并没有简单的将user最近一次购买的item共现系数最高的前50推荐，而是考虑了user历史购买过的所有item，把与这些item共现系数最高的topk=500个item都找出来，然后取出其中 user没有购买过的 共现系数最高的前50个做推荐，如果不足50个，则用所有点击中最高的前50item补足



## 问题：
1. 没有考虑user，即u1点击i1和u2点击i1的推荐结果一样
baseline0_itemCF
2. 没有考虑user的历史点击行为，即u1在点击i1之前点击了i2或者i3的推荐结果一样
baseline0_itemCF
3. 没有考虑user自身特征
baseline0_itemCF
baseline1_itemCF
4. 没有考虑item自身特征
baseline0_itemCF
baseline1_itemCF
5. 没有考虑user对item的点击次数影响，即u1对i1点击过1次、对i2点击过1次和u2对i1点击过100次、对i2点击过100次，在计算i1和i2的共现系数时效果相同
baseline1_itemCF


## 待改进
### baseline0_itemCF
建立 user-item矩阵，
点击过程需要做平滑，如u1对i1点击1次，和u1对i1点击10000次要做平滑，如在计算完点击次数后，取log(cnt+1)，这样，点击0次，为0，点击>0次，结果大于0

推荐过程可以参照baseline1_itemCF的做改进
取出历史购买过的所有item，然后再在里面取出前50做推荐，而不是只考虑最近点击过的一个item

### baseline1_itemCF
推荐过程 改进
1. item添加权重 done  现在第一次点击的item是最后一次点击item权重的0.5，但是可以调优  
分数明显改善！  
如一个用户共点击过100个item，那么最开始点击的item1，肯定要比最近点击的item100对推荐的影响要小，因此item1的相似的topk个物品的权重要小于item100的相似的topk个物品的权重
本来想着可以用time作为权重，但是它们的差异实在太小了！
算了，自己造一个把，最开始点击的item的权重是0.5，最近一次点击的item权重取1，如果item被user点击了多次，则去最近点击的那次

In [11]:
#!/usr/bin/env python  
# -*- coding:utf-8 -*-  
  
import pandas as pd  
from tqdm import tqdm  
from collections import defaultdict  
import math  
  
def get_sim_item(df, user_col, item_col, use_iif=False): 
    """
    use_iif:
        True: 把用户点击过的item总数考虑进去，
            如果用户1和用户2都点击了item[1, 2]，但是用户1一共就点击过10个item，而用户2一共点击过10000个item
            那么在计算item[1, 2]的共现系数时，用户1肯定比用户2的价值大
    """
    user_item_ = df.groupby(user_col)[item_col].agg(set).reset_index()  
    user_item_dict = dict(zip(user_item_[user_col], user_item_[item_col]))  
  
    sim_item = {}  # sim_item[i][j] 表示item_i和item_j共现的次数
    item_cnt = defaultdict(int)  # 统计某个item有多少个user购买过
    for user, items in tqdm(user_item_dict.items()):  
        for i in items:  
            item_cnt[i] += 1  
            sim_item.setdefault(i, {})  
            for relate_item in items:  
                if i == relate_item:  
                    continue  
                sim_item[i].setdefault(relate_item, 0)  
                if not use_iif:  
                    sim_item[i][relate_item] += 1  
                else:  
                    sim_item[i][relate_item] += 1 / math.log(1 + len(items))  
    
    sim_item_corr = sim_item.copy()  
    for i, related_items in tqdm(sim_item.items()):  
        for j, cij in related_items.items():  
            # item_i和item_j共现的次数/各自出现的次数之积
            sim_item_corr[i][j] = cij/math.sqrt(item_cnt[i]*item_cnt[j])  
  
    return sim_item_corr, user_item_dict  
  

def recommend(sim_item_corr, user_item_dict, user_id, top_k, item_num):  
    """
    向user_id推荐item_num个该user_id没有买过的item
    """
    rank = {}  
    # 该user_id购买过的items
#     interacted_items = user_item_dict[user_id]  
    dft = whole_click[whole_click.user_id == user_id].sort_values('time').drop_duplicates('item_id', keep='last')
    dft['t'] = range(dft.shape[0], dft.shape[0] * 2)
    interacted_items = dft['item_id'].tolist()
    weights = dft['t'].tolist()
#     print(dft.shape[0], dft.head())
    # 遍历该user购买过的items，
    cnt = 0
    for i in interacted_items:  
        # 遍历该item共现最高的top_k个item，把其中用户没有买过的加入推荐列表
#         for j, wij in sorted(sim_item_corr[i].items(), reverse=True)[:top_k]:  
        for j, wij in sorted(sim_item_corr[i].items(), key=lambda d: d[1], reverse=True)[:top_k]:  
            if j not in interacted_items:  
                rank.setdefault(j, 0)  
                rank[j] += wij * weights[cnt]
        cnt += 1
    return sorted(rank.items(), key=lambda d: d[1], reverse=True)[:item_num]  
  
def get_predict(df, pred_col, top_fill):  
    """
    fill user to 50 items
    逻辑就是如果推荐给用户的items少于50个，就用点击数最高的item补足
    """
    top_fill = [int(t) for t in top_fill.split(',')]  
    scores = [-1 * i for i in range(1, len(top_fill) + 1)]  
    ids = list(df['user_id'].unique())  
    fill_df = pd.DataFrame(ids * len(top_fill), columns=['user_id'])  
    fill_df.sort_values('user_id', inplace=True)  
    fill_df['item_id'] = top_fill * len(ids)  
    fill_df[pred_col] = scores * len(ids)  
    df = df.append(fill_df)  
    df.sort_values(pred_col, ascending=False, inplace=True)  
    df = df.drop_duplicates(subset=['user_id', 'item_id'], keep='first')  
    df['rank'] = df.groupby('user_id')[pred_col].rank(method='first', ascending=False)  
    df = df[df['rank'] <= 50]  
    df = df.groupby('user_id')['item_id'].apply(lambda x: ','.join([str(i) for i in x])).str.split(',', expand=True).reset_index()  
    return df  

In [12]:
now_phase = 2  
train_path = './data_origin/underexpose_train'  
test_path = './data_origin/underexpose_test'  
recom_item = []  

whole_click = pd.DataFrame()  
click_test = pd.DataFrame()  
for c in range(now_phase + 1):  
    print('phase:', c)  
    click_train = pd.read_csv(train_path + '/underexpose_train_click-{}.csv'.format(c), header=None,  names=['user_id', 'item_id', 'time'])  
    click_test1 = pd.read_csv(test_path + '/underexpose_test_click-{}/underexpose_test_click-{}.csv'.format(c, c), header=None,  names=['user_id', 'item_id', 'time'])  

    all_click = click_train.append(click_test1)  
    click_test = click_test.append(click_test1) 
    whole_click = whole_click.append(all_click)  
    
item_sim_list, user_item = get_sim_item(whole_click, 'user_id', 'item_id', use_iif=True)  
# item_sim_list, user_item = get_sim_item(whole_click, 'user_id', 'item_id', use_iif=False)  

for i in tqdm(click_test['user_id'].unique()):  
    rank_item = recommend(item_sim_list, user_item, i, 500, 50)  
    for j in rank_item:  
        recom_item.append([i, j[0], j[1]])  
# find most popular items  
top50_click = whole_click['item_id'].value_counts().index[:50].values  
top50_click = ','.join([str(i) for i in top50_click])  

recom_df = pd.DataFrame(recom_item, columns=['user_id', 'item_id', 'sim'])  
result = get_predict(recom_df, 'sim', top50_click)  
result.to_csv('baseline.csv', index=False, header=None)

phase: 0
phase: 1
phase: 2


  0%|          | 12/5079 [00:00<01:31, 55.62it/s]

61        user_id  item_id      time   t
15807     1133    87266  0.983790  61
549       1133   104098  0.983791  62
5368      1133    44994  0.983791  63
5247      1133    20158  0.983796  64
2512      1133      255  0.983796  65
20        user_id  item_id      time   t
6158     17864    71401  0.983749  20
14955    17864    49396  0.983749  21
15008    17864    59346  0.983750  22
20352    17864    67454  0.983750  23
20771    17864    30815  0.983750  24
10        user_id  item_id      time   t
5137      6941      237  0.983785  10
6397      6941      220  0.983785  11
2         6941      309  0.983785  12
19373     6941     7111  0.983787  13
20700     6941    14828  0.983787  14
10        user_id  item_id      time   t
2169     34089    46901  0.983748  10
6098     34089    59267  0.983749  11
11529    34089     5726  0.983749  12
3        34089      358  0.983781  13
19955    34089     2589  0.983791  14
50         user_id  item_id      time   t
3665      21659    89195  0.983773

  0%|          | 21/5079 [00:00<01:46, 47.61it/s]

20        user_id  item_id      time   t
1111     24079    92773  0.983775  20
801      24079    29001  0.983789  21
13359    24079   110080  0.983789  22
13672    24079    55257  0.983789  23
17242    24079    99952  0.983817  24
30         user_id  item_id      time   t
6012      15521    43521  0.983773  30
118       15521    22064  0.983774  31
15578     15521    43278  0.983815  32
112441    15521    35669  0.983846  33
219480    15521    84497  0.983869  34
82        user_id  item_id     time   t
19136    20768    79739  0.98378  82
16167    20768    31652  0.98378  83
5572     20768    82317  0.98378  84
15847    20768    95191  0.98378  85
6909     20768    94595  0.98378  86
51         user_id  item_id      time   t
9289       4136    65128  0.983812  51
236116     4136    19207  0.983812  52
171052     4136    34891  0.983812  53
177317     4136    18895  0.983813  54
6219       4136    82154  0.983813  55
22         user_id  item_id      time   t
185432    13805    32291  0.

...

8        user_id  item_id      time   t
12361      409   113026  0.983895   8
21509      409    20158  0.983896   9
17145      409      265  0.983904  10
15612      409    92849  0.984028  11
9612       409      230  0.984028  12
13         user_id  item_id      time   t
127999    14643    50587  0.983754  13
196955    14643   104430  0.983754  14
73029     14643   111932  0.983754  15
14387     14643   112054  0.983787  16
18822     14643    61899  0.983787  17
2        user_id  item_id      time  t
9833      3544    31008  0.984035  2
20149     3544    24744  0.984059  3
12         user_id  item_id      time   t
97097      1872    25217  0.983837  12
13567      1872    97763  0.983866  13
16902      1872    74779  0.983866  14
105590     1872    42435  0.983870  15
150008     1872    52916  0.983870  16
9         user_id  item_id      time   t
238041    34608    56307  0.983784   9
129287    34608    72545  0.983786  10
85817     34608    82450  0.983787  11
21507     34608    20055 

100%|█████████▉| 5060/5079 [01:16<00:00, 104.94it/s]

3        user_id  item_id      time  t
19817    25214    86744  0.983901  3
12061    25214    51566  0.983902  4
19147    25214    92183  0.983902  5
5         user_id  item_id      time  t
233344     7999   100990  0.983749  5
15701      7999    13907  0.983752  6
12259      7999    91093  0.983912  7
12090      7999    57888  0.983914  8
13253      7999    42369  0.984004  9
3        user_id  item_id      time  t
14047    35323    61299  0.983947  3
84918    35323    62441  0.983948  4
12201    35323    80784  0.983990  5
4         user_id  item_id      time  t
153279     2829   105047  0.983846  4
17131      2829   113044  0.983938  5
12347      2829   109913  0.983938  6
43988      2829   109861  0.983942  7
4         user_id  item_id      time  t
214273     9429    27026  0.983775  4
144700     9429    86966  0.983939  5
13261      9429    43067  0.983939  6
12413      9429     6158  0.984058  7
17         user_id  item_id      time   t
154610     5172    10086  0.983742  17
15136

100%|██████████| 5079/5079 [01:16<00:00, 66.24it/s] 


4        user_id  item_id      time  t
49122    29262    75196  0.983870  4
15703    29262   105933  0.983902  5
15694    29262   104588  0.983933  6
18343    29262    78482  0.983943  7
2        user_id  item_id      time  t
18674    24312    14631  0.983987  2
16534    24312    11081  0.984046  3
3        user_id  item_id      time  t
22669     4875   100914  0.983866  3
16552     4875    13336  0.983866  4
17590     4875    74523  0.983889  5
6         user_id  item_id      time   t
43719      2180   105403  0.983826   6
164198     2180    45010  0.983833   7
139677     2180     5993  0.983844   8
26652      2180    67926  0.983951   9
22513      2180    76755  0.983953  10
4         user_id  item_id      time  t
17230     11024    12940  0.983965  4
19329     11024     3693  0.983966  5
16922     11024    77960  0.983973  6
236658    11024    26838  0.983978  7
3        user_id  item_id      time  t
18353    22244    80713  0.983867  3
17267    22244    18386  0.983867  4
49088    